In [ ]:
# import os
# import requests

# # Define parameters for the API request
# params = {
#     'query': 'species:Passer montanus',  # eurasian tree sparrow
#     'page': 1,  # Page number
#     'format': 'json'  # Response format
# }

# # Make the API request
# response = requests.get('https://www.xeno-canto.org/api/2/recordings', params=params)

# # if successful
# if response.status_code == 200:
#     data = response.json()
#     # Process the data, download audio files, etc.
#     for record in data['recordings']:
#         audio_url = record['file']
#         # Extract filename from the URL
#         filename = os.path.basename(audio_url)
#         # Specify the local file path where you want to save the audio file
#         local_filepath = os.path.join('C:\Users\Bojo\Desktop\python projects', filename)
#         # Download the audio file
#         with open(local_filepath, 'wb') as f:
#             audio_response = requests.get(audio_url)
#             f.write(audio_response.content)
#     print("Audio files downloaded successfully.")
# else:
#     print('Error:', response.status_code)

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
from pathlib import Path
import sys
import os
import time
import warnings
# suppress all warnings
warnings.filterwarnings("ignore")

from scipy import signal
import librosa
import librosa.display
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from maad import sound, util, rois

def grab_audio(path, audio_format='mp3'):
    filelist = []
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if name[-3:].casefold() == audio_format and name[:2] != '._':
                filelist.append(os.path.join(root, name))
    return filelist

XC_ROOTDIR = './dataset/'

XC_DIR = 'data_sets'

data = [['Eurasian Tree Sparrow', 'Passer montanus'],
        ['Zebra Dove', 'Geopelia striata'] ]

df_species = pd.DataFrame(data,columns =['english name',
                                         'scientific name'])
gen = []
sp = []
for name in df_species['scientific name']:
    gen.append(name.rpartition(' ')[0])
    sp.append(name.rpartition(' ')[2])

df_query = pd.DataFrame()
df_query['param1'] = gen
df_query['param2'] = sp
df_query['param3'] ='type:distress call'
df_query['param4'] ='area:asia'
df_query['param5'] ='len:"5-120"'
df_query['param5'] ='q:">C"'

# Get recordings metadata corresponding to the query
df_dataset= util.xc_multi_query(df_query,
                                 format_time = False,
                                 format_date = False,
                                 verbose=True)



df_dataset = util.xc_selection(df_dataset,
                               max_nb_files=20,
                               max_length='01:00',
                               min_length='00:10',
                               min_quality='B',
                               verbose = True )

util.xc_download(df_dataset,
                 rootdir = XC_ROOTDIR,
                 dataset_name= XC_DIR,
                 overwrite=False,
                 save_csv= True,
                 verbose = True)

filelist = grab_audio(XC_ROOTDIR+XC_DIR)

df = pd.DataFrame()
for file in filelist:
    df = df.append({'fullfilename': file,
                    'filename': Path(file).parts[-1][:-4],
                    'species': Path(file).parts[-2]},
                   ignore_index=True)

print('=====================================================')
print('number of files : %2.0f' % len(df))
print('number of species : %2.0f' % len(df.species.unique()))
print('=====================================================')
